In [1]:
from __future__ import print_function
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
from tensorboardX import SummaryWriter

from tqdm import tqdm

batch_size = 128
epochs = 100
seed = 1
torch.manual_seed(seed)

if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

In [2]:
from Tars.distributions import Deterministic, DataDistribution
from Tars.distributions import Normal
from Tars.models import GAN

In [3]:
kwargs = {'num_workers': 1, 'pin_memory': True}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True, **kwargs)

In [4]:
x_dim = 784
z_dim = 64

# generator model p(x|z)    
class Generator(Deterministic):
    def __init__(self):
        super(Generator, self).__init__(cond_var=["z"], var=["x"], name="p")

        self.fc1 = nn.Linear(z_dim, 512)
        self.fc2 = nn.Linear(512, 512)
        self.fc3 = nn.Linear(512, x_dim)
        self.bn1 = nn.BatchNorm1d(512)
        self.bn2 = nn.BatchNorm1d(512)        

    def forward(self, z):
        h = F.relu(self.bn1(self.fc1(z)))
        h = F.relu(self.bn2(self.fc2(h)))
        return {"x": F.sigmoid(self.fc3(h))}
    
    
# prior model p(z)
loc = torch.tensor(0.).to(device)
scale = torch.tensor(1.).to(device)
prior = Normal(loc=loc, scale=scale, var=["z"], dim=z_dim, name="p_prior")

In [5]:
# generative model
p_g = Generator()
p = (p_g*prior).marginalize_var("z")
p.to(device)

# data distribution
p_data = DataDistribution(["x"])
p_data.to(device)

print(p)
print(p_data)

Distribution:
  p(x) = ∫p(x|z)p_prior(z)dz
Network architecture:
  p_prior(z) (Normal): Normal()
  p(x|z) (Deterministic): Generator(
    (fc1): Linear(in_features=64, out_features=512, bias=True)
    (fc2): Linear(in_features=512, out_features=512, bias=True)
    (fc3): Linear(in_features=512, out_features=784, bias=True)
    (bn1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
Distribution:
  p_data(x) (Data distribution)
Network architecture:
  DataDistribution()


In [6]:
# discriminator model p(t|x)
class Discriminator(Deterministic):
    def __init__(self):
        super(Discriminator, self).__init__(cond_var=["x"], var=["t"], name="d")

        self.fc1 = nn.Linear(x_dim, 512)
        self.fc2 = nn.Linear(512, 512)
        self.fc3 = nn.Linear(512, 1)

    def forward(self, x):
        h = F.leaky_relu(self.fc1(x))
        h = F.leaky_relu(self.fc2(h))
        return {"t": F.sigmoid(self.fc3(h))}
    
d = Discriminator()
d.to(device)

print(d)

Distribution:
  d(t|x) (Deterministic)
Network architecture:
  Discriminator(
    (fc1): Linear(in_features=784, out_features=512, bias=True)
    (fc2): Linear(in_features=512, out_features=512, bias=True)
    (fc3): Linear(in_features=512, out_features=1, bias=True)
  )


In [7]:
model = GAN(p_data, p, d,
            optimizer=optim.Adam, optimizer_params={"lr":1e-4},
            d_optimizer=optim.Adam, d_optimizer_params={"lr":1e-4})
print(model)

Distributions (for training): 
  p(x) 
Loss function: 
  mean(GANLoss[p_data(x)||p(x)])


In [8]:
def train(epoch):
    train_loss = 0
    train_d_loss = 0
    for batch_idx, (data, _) in enumerate(tqdm(train_loader)):
        data = data.to(device)
        loss, d_loss = model.train({"x": data.view(-1, 784)})
        train_loss += loss
        train_d_loss += d_loss
 
    train_loss = train_loss * train_loader.batch_size / len(train_loader.dataset)
    train_d_loss = train_d_loss * train_loader.batch_size / len(train_loader.dataset)
    print('Epoch: {} Train loss: {:.4f}, {:.4f}'.format(epoch, train_loss.item(), train_d_loss.item()))
    return train_loss

In [9]:
def test(epoch):
    test_loss = 0
    test_d_loss = 0
    for i, (data, _) in enumerate(test_loader):
        data = data.to(device)
        loss, d_loss = model.test({"x": data.view(-1, 784)})
        test_loss += loss
        test_d_loss += d_loss

    test_loss = test_loss * test_loader.batch_size / len(test_loader.dataset)
    test_d_loss = test_d_loss * test_loader.batch_size / len(test_loader.dataset)
    
    print('Test loss: {:.4f}, {:.4f}'.format(test_loss, test_d_loss.item()))
    return test_loss

In [10]:
def plot_image_from_latent(z_sample):
    with torch.no_grad():
        sample = p_g.sample({"z": z_sample})["x"].view(-1, 1, 28, 28).cpu()
        return sample

In [11]:
writer = SummaryWriter()

z_sample = torch.randn(64, z_dim).to(device)
x_original, y_original = iter(test_loader).next()
x_original = x_original.to(device)
y_original = y_original.to(device)

for epoch in range(1, epochs + 1):
    train_loss = train(epoch)
    test_loss = test(epoch)
    
    sample = plot_image_from_latent(z_sample)

    writer.add_scalar('train_loss', train_loss.item(), epoch)
    writer.add_scalar('test_loss', test_loss.item(), epoch)      
    
    writer.add_image('Image_from_latent', sample, epoch)
    
writer.close()

100%|██████████| 469/469 [00:05<00:00, 85.21it/s]

Epoch: 1 Train loss: 4.8727, 0.1210



  0%|          | 0/469 [00:00<?, ?it/s]

Test loss: 5.7470, 5.7492


100%|██████████| 469/469 [00:05<00:00, 84.07it/s]


...


  0%|          | 0/469 [00:00<?, ?it/s]

Test loss: 2.8283, 2.8097


100%|██████████| 469/469 [00:05<00:00, 83.44it/s]

Epoch: 100 Train loss: 2.5926, 0.5962


Test loss: 2.9826, 2.9595
